In [ ]:
import warnings; warnings.filterwarnings('ignore')
from keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
from keras.applications import imagenet_utils, VGG19 #Xception, InceptionV3, MobileNetV2
from uuid import NAMESPACE_URL, uuid1, uuid3
from keras.models import load_model
from skimage.color import rgb2gray
from traceback import format_exc
import matplotlib.pyplot as plt
from functools import lru_cache
from annoy import AnnoyIndex
from six import string_types
from random import randint
import keras.backend as K
from copy import deepcopy
from hashlib import sha1
from sys import maxsize
from math import ceil
import numpy as np
import skimage
import glob, json, shears, os

NB: This pipeline is founded on the idea that each image is composed of 0 or more "figures",
where each figure is some semantic unit within the image (e.g. a single plant, or part of a plant).

To clear all caches, remove `meta_d.json`, `./ann`, and `voynich.ann`.

TODOS:
 * quantize npy vecs for faster i/o and reduced disk usage
 * suture individual metadata from the data repo to meta_d.json

# Helpers

In [ ]:
##
# NN MODEL
##

# VGG16, VGG19, and ResNet take 224×224 images; InceptionV3 and Xception take 299×299 inputs
def resize_array(arr, shape=(304,304)):
  '''Resize an array to a new shape'''
  im = array_to_img(arr)
  resized = im.resize(shape)
  return img_to_array(resized)/255.0


def get_uuid(*args, dtype='str', max_size=maxsize, right_pad=True):
  '''Helper method that returns a random integer'''
  if args and isinstance(args[0], string_types):
    if dtype == 'int': i = str(get_int_hash(args[0]))
    elif dtype == 'str': i = str(uuid3(NAMESPACE_URL, args[0]))
  else:
    if dtype == 'int': i = str(randint(0, max_size))
    elif dtype == 'str': i = str(uuid1())
  if dtype == 'int':
    if right_pad:
      while len(i) < len(str(max_size)): i = i + '0'
    return int(i)
  return i


def get_int_hash(s):
  '''Given a string return an integer hash of the string's content'''
  try:
    return int(sha1(args[0]).hexdigest(), 16)
  except:
    return int(sha1(args[0].encode('utf8')).hexdigest(), 16)


def fit_transform(obj, sample_layer_index=-1):
  # if the input array is empty, skip
  if not np.any(obj): return False
  if obj.shape == (0,): return None
  # input shape must be n_images, h, w, colors: https://keras.io/preprocessing/image/
  if hasattr(obj, 'shape') and len(obj.shape) == 3:
    # this is an array of a single image
    arr = resize_array(obj)
    arr = np.expand_dims(arr, axis=0)
  elif hasattr(obj, 'shape') and len(obj.shape) == 4:
    # this is an array of images
    arr = np.array([resize_array(i) for i in obj])
  else:
    # this is assumed to be a single image
    arr = resize_array(img_to_array(obj))/255.0 # see SO 47697622
  # only Xception requires preprocessing
  arr = imagenet_utils.preprocess_input(arr)
  # extract the ith layer from the model (here, the -1th layer, or final layer)
  out = K.function([model.input], [model.layers[sample_layer_index].output])([arr])
  # parse out the k dim vector (k is determined by the size of the layer that's sampled from)
  vec = out[0]
  # return the vector to the calling scope
  return vec


##
# IMAGE CLEANING
##

@lru_cache(maxsize=0)
def load_image(path, color_mode='rgb'):
  '''Given the path to an image return a keras image object'''
  # nb all image loading should call this method rather than load_img
  im = img_to_array(load_img(path, color_mode=color_mode))/255.0
  if color_mode == 'grayscale': im = img_to_grayscale(im)
  im = resize_image(im)
  return im


def resize_image(im, width=1024):
  '''Given an image array, resize that image to a target width while maintaining aspect ratio'''
  h,w = im.shape[:2]
  height = h/w*width
  resized = img_to_array( array_to_img(im).resize( (int(width), int(height) )) )/255.0
  return resized
  
  
def clean_image(im):
  '''Given an image array return an image array that has standard width and non-significant pixels removed'''
  im = resize_image(im)
  _im = shears.remove_dominant_colors(im, mask_size=0.05, n_colors_to_remove=2)
  _im = shears.filter_img(_im, min_size=1200, connectivity=60) # binarized mask-like
  # create a mask from the remaining dark pixels in _im
  im[np.where(_im==1)] = 1
  return im


##
# IMAGE PARTITIONING
##

@lru_cache(maxsize=0)
def get_image_figures(img_path, grayscale=True, resize_figures=True, min_area=10000):
  '''
  Given the path to an image, return an iterable of numpy arrays, with one
  for each "figure" in the image, where a figure represents a single
  object that's figured, or represented (like a plant, or planet)
  ''' 
  orig = resize_image(load_image(img_path)) # get image
  gray = skimage.color.rgb2gray(orig) # grayscale
  o = np.zeros(gray.shape) # binarize
  o[np.where(gray>skimage.filters.threshold_otsu(gray))] = 1 # binarize
  o = shears.filter_img(o, min_size=500, connectivity=60) # remove text
  o = 1-o # reverse figure / ground
  l = [] # initialize list that will hold extracted figures
  for i in skimage.measure.regionprops(skimage.measure.label(o, connectivity=2)): # find figures
    if i.area >= min_area:
      y_min, x_min, y_max, x_max = i.bbox
      # quick gut check to make sure this patch is not abnormally 
      w = x_max-x_min
      h = y_max-y_min
      if (w/h<0.2) or (h/w<0.2): continue
      img = img_to_grayscale(orig) if grayscale else orig
      l.append(img[y_min:y_max, x_min:x_max])
  if resize_figures:
    return np.array([resize_array(i) for i in l])
  return np.array([i for i in l])


@lru_cache(maxsize=0)
def get_image_windows(img_path, color_mode='rgb'):
  '''Given the path to an image, return an array of the image windows within that image'''
  img = load_image(img_path, color_mode=color_mode)
  arr = img_to_array(img)/255.0
  windows = get_windows(arr)
  return windows


def get_windows(im, step=20, win=300, n_per_side=20, plot=False):
  '''
  Given a np array `im` with at least two dimensions, return windows of size (`win`, `win`).
  To create the list of windows, remove text, binarize, then use that binarized result as a 
  mask to remove non-significant pixels from the original image. Finally, pass a sliding window
  over the remainder. Only retain the subset of those windows that contain some minimum amount of
  pixels in the mask.
  '''
  s = win+(step*n_per_side)
  im = resize_array(im, shape=(s,s))
  windows = []
  dx = 0
  dy = 0
  for _ in range(im.shape[0]//step): # x axis pass
    for _ in range(im.shape[1]//step): # y axis pass
      w = im[dy:dy+win, dx:dx+win]
      if w.shape == (win, win, 3):
        windows.append(w)
      if plot:
        plt.imshow(w)
        plt.title('{}-{}'.format(dx, dy))
        plt.show()
      dy += step
    dx += step
  return np.array(windows)


@lru_cache(maxsize=0)
def guid_to_vecs(guid):
  '''Given a guid for a full image, get the array of vectors that corresponds to that image'''
  vec_path = os.path.join(out_dir, '{}-vectors.npy'.format(guid))
  try:
    vecs = np.load(vec_path)
    if np.any(vecs): return vecs
  except:
    print(' ! could not load vecs for guid', guid)
  return []


@lru_cache(maxsize=0)
def guid_to_vec(guid):
  '''Given a guid for a figure, get the vector that corresponds to that figure'''
  m = meta_d[guid]
  return guid_to_vecs(m['parent_guid'])[m['vec_idx']]

##
# PLOTING
##

def plot_img_grid(img_list, rows=1, size_scalar=2, labels=[]):
  '''Plot a list of images with `rows` rows'''
  cols = ceil( len(img_list) / rows )
  # initialize the plot
  fig, axes = plt.subplots(rows, cols, figsize=(cols*size_scalar, rows*size_scalar), squeeze=False)
  for idx, i in enumerate(img_list):
    col = idx%cols
    row = idx//cols
    axes[row][col].imshow(img_list[idx])
    if len(labels) > idx:
      axes[row][col].set_title(labels[idx], fontsize=6.5)
  plt.show()

  
def plot_guid(guid, title=None):
  '''Given a GUID plot the image represented by that GUID'''
  window = guid_to_img(guid)
  title = title if title else guid_to_title(guid)
  plt.imshow(window)
  plt.title(title)
  plt.show()
  
  
def guid_to_img(guid):
  '''Given a GUID return a numpy image'''
  if meta_d['guid'].get('parent_guid', False):
    parent_guid = meta_d['guid']['parent_guid']
    windows = get_image_figures(meta_d[parent_guid]['path'])
    return windows[guid][meta_d['guid']['vec_idx']]
  else:
    return img_to_arry(load_image(meta_d[guid]['path']))
  
  
def guid_to_title(guid):
  '''Given a GUID return a title for the image'''
  m = meta_d[guid]
  if not m.get('parent_guid', False):
    return m.get('path')
  return '{} {} {}'.format(m['collection'], m['idx_in_collection'], m['vec_idx'])


def img_to_grayscale(im):
  '''Given an image with shape h,w,3, return the image with the same shape in grayscale'''
  im = rgb2gray(im).squeeze()
  _im = np.zeros((im.shape[0], im.shape[1], 3))
  # replicate identical color information across all channels
  _im[:,:,0] = im
  _im[:,:,1] = im
  _im[:,:,2] = im
  return _im

# Load Images

Each image contains multiple figures. Extract each...

In [ ]:
images = glob.glob('utils/voynichese/images/*.jpg')

In [ ]:
images += glob.glob('data/uvm-italian-herbal/images/*.jpg') # load other collections...

In [ ]:
# only for development & experimentation
if False:
  np.random.seed(0)
  np.random.shuffle(images)
  images = images[:100]

In [ ]:
figures = [get_image_figures(i) for i in images] # 2D list wherein sublists contain figures for the ith image in images

In [ ]:
plt.hist([len(i) for i in figures]) # histogram of the number of figures in each image
plt.title('Counts of Figures Per Input Image')
plt.xlabel('n Figures')
plt.ylabel('Count')
plt.show()

To keep things easy in subsequent sections, flatten each list of images and map each subimage to an index a-b, where a denotes index of the figure's parent image in all images, and b denotes the index of the figure within all figures in that image

In [ ]:
fig_map = {}
k = 0
for idx, i in enumerate(figures):
  for jdx, j in enumerate(i):
    fig_map[k] = '{}-{}'.format(idx, jdx)
    k += 1

In [ ]:
imgs_color = np.array([j for i in figures for j in i])

In [ ]:
import gc

# free up some memory
del figures

# optionally clear the cache on lru decorated functions
load_image.cache_clear()
get_image_figures.cache_clear()
get_image_windows.cache_clear()

gc.collect()

# Image Vectorization

Next let's transform each image into a vector. There are many methods we can use to achieve this:

1. We can flatten each input image into a vector
2. We can sample from a hidden layer in a pretrained network
3. We can build our own network and vectorize images with that

### Raw Image Vectorization

In [ ]:
z = X = imgs_color

### Vectorize with Pretrained Network

In [ ]:
# VGG16, VGG19, and ResNet take 224×224 images; InceptionV3 and Xception take 299×299 inputs
X = np.array([resize_array(i, shape=(224,224)) for i in imgs_color])

In [ ]:
# specify the model to use and identify the size of its internal layers
#model = Xception()
#model = MobileNetV2()
model = VGG19()

In [ ]:
for idx, i in enumerate(model.layers):
  shape = i.output_shape[1:]
  vals = np.prod(shape)
  print(idx, vals)

In [ ]:
# select the index of the layer to use when vectorizing images
# nb: different layers have different shapes!
sample_layer_index = 6

# only xception requires img preprocessing
if model.name == 'xception': X = imagenet_utils.preprocess_input(X)
  
# vectorization function that extracts the output from the `sample_layer_index` layer
vectorize = K.function([model.input], [model.layers[sample_layer_index].output])
  
# to process the entire dataframe in one shot
if False:
  z = vectorize(X)
  z = np.array([i.flatten() for i in z])
  
# for progress
if True:
  vecs = []
  for idx, i in enumerate(X):
    query_img = np.expand_dims(i, 0)
    vec = vectorize([query_img])[0]
    vecs.append(vec)
    print(' * vectorized', idx+1, 'of', len(X), 'frames')
  z = np.array([i.flatten() for i in vecs])

In [ ]:
# scale X 0:1 so images can be visualized below
X = rgb2gray(X)
X = X-np.min(X)
X /= np.max(X)

In [ ]:
np.save('z',z)
np.save('X',X)
np.save('images',images)
json.dump(fig_map, open('fig_map.json', 'w'))

### Vectorize with a Custom CNN

In [ ]:
from keras.models import Model
from keras.layers import Input, Reshape, Dense, Flatten, Conv2D, MaxPooling2D, UpSampling2D

conv1 = (6,6)
conv2 = (5,5)
conv3 = (4,4)
conv4 = (4,4)

pool1 = (2,2)
pool2 = (2,2)
pool3 = (2,2)
pool4 = (2,2)

class Autoencoder:
  def __init__(self, img_shape=(304, 304, 1)):
    if not img_shape: raise Exception('Please provide img_shape (height, width) in px')

    # create the encoder
    i = h = Input(img_shape) # the encoder takes as input images    
    h = Conv2D(16, conv1, activation='relu', padding='same')(i)
    h = MaxPooling2D(pool1, padding='same')(h)
    h = Conv2D(8, conv2, activation='relu', padding='same')(h)
    h = MaxPooling2D(pool2, padding='same')(h)
    h = Conv2D(8, conv3, activation='relu', padding='same')(h)
    h = MaxPooling2D(pool3, padding='same')(h)
    h = Conv2D(4, conv4, activation='relu', padding='same')(h)
    h = MaxPooling2D(pool4, padding='same')(h)
    self.encoder = Model(inputs=[i], outputs=[h])

    # create the decoder
    i = h = Input((19, 19, 4))
    h = Conv2D(4, conv4, activation='relu', padding='same')(h)
    h = UpSampling2D(pool4)(h)
    h = Conv2D(8, conv3, activation='relu', padding='same')(h)
    h = UpSampling2D(pool3)(h)
    h = Conv2D(8, conv2, activation='relu', padding='same')(h)
    h = UpSampling2D(pool2)(h)
    h = Conv2D(16, conv1, activation='relu', padding='same')(h)
    h = UpSampling2D(pool1)(h)
    h = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(h)
    self.decoder = Model(inputs=[i], outputs=[h])

    # combine the encoder and decoder into a full autoencoder
    i = Input(img_shape) # take as input image vectors
    z = self.encoder(i) # push observations into latent space
    o = self.decoder(z) # project from latent space to feature space
    self.model = Model(inputs=[i], outputs=[o])
    self.model.compile(loss='mse', optimizer='adam')

autoencoder = Autoencoder()

# inspect the model
autoencoder.encoder.summary()
autoencoder.decoder.summary()
autoencoder.model.summary()

In [ ]:
# this autoencoder expects a single color channel (i.e. grayscale inputs)
X = np.expand_dims(np.array([rgb2gray(i) for i in imgs_color]), 3)

In [ ]:
# train the autoencoder - for production purposes run more epochs, ~100 is a decent start.
# once the loss term ceases to fall much over many iterations,
# you can lower the learning rate and train more...
autoencoder.model.fit(X, X, batch_size=256, epochs=10)

In [ ]:
# optionally, save or load a model -- useful if you run lots of training
if False:
  model.save('voynich.hdf5')
if False:
  model = load_model('voynich.hdf5')

In [ ]:
# test autoencoder's reconstruction of sample inputs
# the closer outputs are to the input, the better the model
# has learned the data's features

# select the index of the image to test with
sample_idx = 131

# plot the model input
plt.imshow(X[sample_idx].squeeze())
plt.show()

# plot the model output
o = autoencoder.model.predict(np.expand_dims(X[sample_idx], 0))
plt.imshow(o.squeeze())

# Identify KNN

Whichever vectorization method we use, let's now identify the k-nearest neighbors for each of our figures.

In [ ]:
z.shape, z[0].flatten().shape

In [ ]:
# Build a KNN index
from annoy import AnnoyIndex

build = True

if build:
  t = AnnoyIndex(len(z[0].flatten()), 'angular')
  for idx, i in enumerate(z):
    t.add_item(idx, i.flatten())
  _ = t.build(1000)
  t.save('voynich.ann')
  
else:
  t = AnnoyIndex(len(z[0].flatten()), 'angular')
  model = t.load('test.ann')

In [ ]:
# number of nearest neighbors to find (the first knn is the query image itself usually)
k = 4

for i in range(len(z)):
  knn, dist = t.get_nns_by_item(i, k, include_distances=True)
  sims = [round(1-i, 4) for i in dist]
  if max(sims[1:]) < 0.8: continue
  img_list = [X[knn[i]].squeeze() for i in range(len(knn))] # show the most similar image to the input img
  # curate titles for each match
  titles = []
  img_idx, fig_idx = zip(*[[int(j) for j in fig_map[i].split('-')] for i in knn])  
  print('\n'.join(['{} {} {}'.format(knn[idx], sims[idx], images[i]) for idx, i in enumerate(img_idx)]))
  plot_img_grid(img_list, labels=sims)